In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import csv

from statistics import mean
from keras.layers import Input, Dense
from keras.models import Model
from numpy import linalg as LA
import sklearn
from sklearn import metrics

Using TensorFlow backend.


In [3]:
def pad(l, content, width):
    l.extend([content] * (width - len(l)))
    return l

def trunct(l,width):
    for i in range(0,len(l)-width):
        l.pop()
    return l    

# Retrieving samples for each emotion
usr_dir="Data_files/"
for user in range(1,23,1):
    usr_file=usr_dir+"U"+str(user)+"_LIHF_itd_file.csv"
    
    
    happy_bucket=[]
    sad_bucket=[]
    stressed_bucket=[]
    relax_bucket=[]
    
    print(usr_file)
    dataset=pd.read_csv(usr_file,header=None)
    dataframe=dataset.values
    
    #print("row_number="+str(dataframe.shape[0]))
    
    # value list holds unique id 
    value_list=[]
    for i in range(dataframe.shape[0]):
        value_list.append(dataframe[i][0])
        
    ID_list=list(set(value_list))
    ID_list.sort()
    print("number of unique ID="+str(len(ID_list)))
    start=0
    next_start=0
    # traverse the whole list and add list of ITD values in emotion list
    for i in range(len(ID_list)):
        ID_number=ID_list[i]
        #print("ID_number="+str(ID_number))
        ITD_values=[]
        
        for j in range(start,dataframe.shape[0],1):
            if(dataframe[j][0]==ID_number):
                ITD_values.append(dataframe[j][2])
                next_start=next_start+1
            else:
                start=next_start
                #print(next_start)
                break
        if(dataframe[j-1][4]==2 and len(ITD_values)>3):
             happy_bucket.append(list(ITD_values))
        if(dataframe[j-1][4]==-2 and len(ITD_values)>3):
             sad_bucket.append(list(ITD_values))
        if(dataframe[j-1][4]==1 and len(ITD_values)>3):
             stressed_bucket.append(list(ITD_values))
        if(dataframe[j-1][4]==0 and len(ITD_values)>3):
            relax_bucket.append(list(ITD_values))
        #print("ID_number="+str(ID_number)+","+"length of list="+str(len(ITD_values)))
    print("Total number of samples/buckets="+str(len(happy_bucket)+len(sad_bucket)+len(stressed_bucket)+len(relax_bucket)))
    count_ITD_list=len(happy_bucket)+len(sad_bucket)+len(stressed_bucket)+len(relax_bucket)
    
    # extending all samples to the length of sample with maximum length
    
    length_lists=[]
    
    for itd_list in happy_bucket:
        length_lists.append(len(itd_list))
    #print(length_lists)  
    for itd_list in sad_bucket:
        length_lists.append(len(itd_list))
    for itd_list in stressed_bucket:
        length_lists.append(len(itd_list))  
    for itd_list in relax_bucket:
        length_lists.append(len(itd_list))  
    
    # this is to make a list to the length of list with maximum length/average length
    print("length of maximum list="+str(max(length_lists)))
    #max_length=max(length_lists) 
    avg_length=round(mean(length_lists))
    
    for i in range(len(happy_bucket)):
        if(len(happy_bucket[i])>avg_length):
            happy_bucket[i]=trunct(happy_bucket[i],avg_length)
        elif(len(happy_bucket[i])<avg_length):    
            happy_bucket[i]=pad(happy_bucket[i],0,avg_length)
            
        #happy_bucket[i]=pad(happy_bucket[i],0,max_length)
        # append label
        happy_bucket[i].append(2)
        
    for i in range(len(sad_bucket)):
        if(len(sad_bucket[i])>avg_length):
            sad_bucket[i]=trunct(sad_bucket[i],avg_length)
        elif(len(sad_bucket[i])<avg_length):    
            sad_bucket[i]=pad(sad_bucket[i],0,avg_length)
        #sad_bucket[i]=pad(sad_bucket[i],0,max_length)
        # append label
        sad_bucket[i].append(-2)
    
    for i in range(len(stressed_bucket)):
        if(len(stressed_bucket[i])>avg_length):
            stressed_bucket[i]=trunct(stressed_bucket[i],avg_length)
        elif(len(stressed_bucket[i])<avg_length):    
            stressed_bucket[i]=pad(stressed_bucket[i],0,avg_length)
        #stressed_bucket[i]=pad(stressed_bucket[i],0,max_length)
        # append label
        stressed_bucket[i].append(1)
        
    for i in range(len(relax_bucket)):
        if(len(relax_bucket[i])>avg_length):
            relax_bucket[i]=trunct(relax_bucket[i],avg_length)
        elif(len(relax_bucket[i])<avg_length):    
            relax_bucket[i]=pad(relax_bucket[i],0,avg_length)
        #relax_bucket[i]=pad(relax_bucket[i],0,max_length)
        # append label
        relax_bucket[i].append(0)  
        
    ######### writting itd_list in a file for further processing #################################    
    raw_file="Raw_data/ITD_list_avg/"+"U"+str(user)+"_LIHF_itd_file.csv"  
    with open(raw_file,"w") as resultfile:
        writer=csv.writer(resultfile)
        writer.writerows(happy_bucket)
        writer.writerows(sad_bucket)
        writer.writerows(stressed_bucket)
        writer.writerows(relax_bucket)

Data_files/U1_LIHF_itd_file.csv
number of unique ID=114
Total number of samples/buckets=114
length of maximum list=2539
Data_files/U2_LIHF_itd_file.csv
number of unique ID=45
Total number of samples/buckets=45
length of maximum list=2539
Data_files/U3_LIHF_itd_file.csv
number of unique ID=48
Total number of samples/buckets=45
length of maximum list=339
Data_files/U4_LIHF_itd_file.csv
number of unique ID=118
Total number of samples/buckets=118
length of maximum list=1975
Data_files/U5_LIHF_itd_file.csv
number of unique ID=66
Total number of samples/buckets=66
length of maximum list=2539
Data_files/U6_LIHF_itd_file.csv
number of unique ID=49
Total number of samples/buckets=49
length of maximum list=162
Data_files/U7_LIHF_itd_file.csv
number of unique ID=111
Total number of samples/buckets=111
length of maximum list=227
Data_files/U8_LIHF_itd_file.csv
number of unique ID=27
Total number of samples/buckets=27
length of maximum list=433
Data_files/U9_LIHF_itd_file.csv
number of unique ID=49